In [1]:
#coding=utf-8
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn import preprocessing  
min_max_scaler = preprocessing.MinMaxScaler()  
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import train_test_split

llb


In [3]:
data = pd.read_csv("../add/willnorm/contest_ext_crd_is_sharedebt.csv")
print(data.shape)
print(data.columns)
data.head()

(39948, 64)
Index(['REPORT_ID', 'TYPE_DW_num', 'FINANCE_CORP_COUNT_max',
       'FINANCE_CORP_COUNT_min', 'FINANCE_CORP_COUNT_mean',
       'FINANCE_CORP_COUNT_zongshu', 'FINANCE_CORP_COUNT_num',
       'FINANCE_ORG_COUNT_max', 'FINANCE_ORG_COUNT_min',
       'FINANCE_ORG_COUNT_mean', 'FINANCE_ORG_COUNT_zongshu',
       'FINANCE_ORG_COUNT_num', 'ACCOUNT_COUNT_max', 'ACCOUNT_COUNT_min',
       'ACCOUNT_COUNT_mean', 'ACCOUNT_COUNT_zongshu', 'ACCOUNT_COUNT_num',
       'CREDIT_LIMIT_max', 'CREDIT_LIMIT_min', 'CREDIT_LIMIT_mean',
       'CREDIT_LIMIT_zongshu', 'CREDIT_LIMIT_num',
       'MAX_CREDIT_LIMIT_PER_ORG_max', 'MAX_CREDIT_LIMIT_PER_ORG_min',
       'MAX_CREDIT_LIMIT_PER_ORG_mean', 'MAX_CREDIT_LIMIT_PER_ORG_zongshu',
       'MAX_CREDIT_LIMIT_PER_ORG_num', 'MIN_CREDIT_LIMIT_PER_ORG_max',
       'MIN_CREDIT_LIMIT_PER_ORG_min', 'MIN_CREDIT_LIMIT_PER_ORG_mean',
       'MIN_CREDIT_LIMIT_PER_ORG_zongshu', 'MIN_CREDIT_LIMIT_PER_ORG_num',
       'MAX_CREDIT_LIMIT_PER_ORGCutMIN_CREDIT_LIMIT_

,REPORT_ID,TYPE_DW_num,FINANCE_CORP_COUNT_max,FINANCE_CORP_COUNT_min,FINANCE_CORP_COUNT_mean,FINANCE_CORP_COUNT_zongshu,FINANCE_CORP_COUNT_num,FINANCE_ORG_COUNT_max,FINANCE_ORG_COUNT_min,FINANCE_ORG_COUNT_mean,...,CREDIT_LIMITCutBALANCE_mean,CREDIT_LIMITCutBALANCE_zongshu,CREDIT_LIMITCutBALANCE_num,LATEST_6M_USED_AVG_AMOUNT_max,LATEST_6M_USED_AVG_AMOUNT_min,LATEST_6M_USED_AVG_AMOUNT_mean,LATEST_6M_USED_AVG_AMOUNT_zongshu,LATEST_6M_USED_AVG_AMOUNT_num,TYPE_DW_more,TYPE_DW_less
0,8787.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3435.450620,1.0,1.0,9510.0,9510.0,9510.0,1.0,1.0,1.0,1.0
1,9410.0,2.0,13.0,5.0,9.0,1.0,2.0,13.0,5.0,9.0,...,30318.050000,1.0,2.0,67483.0,6027.0,36755.0,1.0,2.0,1.0,1.0
2,22999.0,3.0,6.0,1.0,3.0,1.0,3.0,6.0,1.0,3.0,...,10426.560396,1.0,3.0,67051.0,437.0,23854.0,1.0,3.0,1.0,1.0
3,24820.0,1.0,8.0,8.0,8.0,1.0,1.0,8.0,8.0,8.0,...,27046.200000,1.0,1.0,62222.0,62222.0,62222.0,1.0,1.0,1.0,1.0
4,25708.0,2.0,4.0,1.0,2.0,1.0,2.0,4.0,1.0,2.0,...,2170.400000,1.0,2.0,26444.0,513.0,13478.5,1.0,2.0,1.0,1.0


In [6]:
data.TYPE_DW_more.unique()

array([1.])

In [4]:
# 6 个类别 *2
label =['TYPE_DW_more', 'TYPE_DW_less']
#
cate = []
c = 0
size = 0
for i in range(len(label)):
    tmp = []
    tmp = pd.get_dummies(data[label[i]],prefix = label[i])
    size += tmp.shape[1]
    if c == 0:
        cate = tmp
        c = 3
        continue
    cate = pd.concat([cate,tmp],axis = 1)
cate.head()

,TYPE_DW_more_1.0,TYPE_DW_less_1.0
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1


In [9]:
label = data.iloc[:,1:-2].columns
# print(label)
number = data.iloc[:,1:-2]
kk = min_max_scaler.fit_transform(number)  
numbernew = pd.DataFrame(kk,columns = label)
numbernew.head()
fulldata = pd.concat([data.REPORT_ID,numbernew],axis = 1)
fulldata.head()

,REPORT_ID,TYPE_DW_num,FINANCE_CORP_COUNT_max,FINANCE_CORP_COUNT_min,FINANCE_CORP_COUNT_mean,FINANCE_CORP_COUNT_zongshu,FINANCE_CORP_COUNT_num,FINANCE_ORG_COUNT_max,FINANCE_ORG_COUNT_min,FINANCE_ORG_COUNT_mean,...,CREDIT_LIMITCutBALANCE_max,CREDIT_LIMITCutBALANCE_min,CREDIT_LIMITCutBALANCE_mean,CREDIT_LIMITCutBALANCE_zongshu,CREDIT_LIMITCutBALANCE_num,LATEST_6M_USED_AVG_AMOUNT_max,LATEST_6M_USED_AVG_AMOUNT_min,LATEST_6M_USED_AVG_AMOUNT_mean,LATEST_6M_USED_AVG_AMOUNT_zongshu,LATEST_6M_USED_AVG_AMOUNT_num
0,8787.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0000,0.000000,0.000000,...,0.002373,0.013965,0.004585,0.0,0.0,0.011383,0.022856,0.022456,0.0,0.0
1,9410.0,0.5,0.800000,0.333333,0.666667,0.0,0.5,0.7500,0.333333,0.666667,...,0.028517,0.069436,0.039047,0.0,0.5,0.080776,0.014485,0.086791,0.0,0.5
2,22999.0,1.0,0.333333,0.000000,0.166667,0.0,1.0,0.3125,0.000000,0.166667,...,0.018201,0.006608,0.013548,0.0,1.0,0.080259,0.001050,0.056327,0.0,1.0
3,24820.0,0.0,0.466667,0.583333,0.583333,0.0,0.0,0.4375,0.583333,0.583333,...,0.018037,0.105185,0.034853,0.0,0.0,0.074479,0.149544,0.146927,0.0,0.0
4,25708.0,0.5,0.200000,0.000000,0.083333,0.0,0.5,0.1875,0.000000,0.083333,...,0.002974,0.000692,0.002964,0.0,0.5,0.031653,0.001233,0.031827,0.0,0.5


In [10]:
#删掉一些维度
train = pd.read_csv("../input/contest_basic_train.tsv",sep='\t')
mm = pd.concat([train.REPORT_ID,train.Y],axis = 1)
ceshidata = fulldata.merge(mm,how = 'inner',left_on = 'REPORT_ID',right_on = 'REPORT_ID') 
print(ceshidata.head())
print(ceshidata.shape)

  REPORT_ID  TYPE_DW_num  FINANCE_CORP_COUNT_max  FINANCE_CORP_COUNT_min  \
0      8787          0.0                0.000000                0.000000   
1      9410          0.5                0.800000                0.333333   
2     22999          1.0                0.333333                0.000000   
3     24820          0.0                0.466667                0.583333   
4     25708          0.5                0.200000                0.000000   

   FINANCE_CORP_COUNT_mean  FINANCE_CORP_COUNT_zongshu  \
0                 0.000000                         0.0   
1                 0.666667                         0.0   
2                 0.166667                         0.0   
3                 0.583333                         0.0   
4                 0.083333                         0.0   

   FINANCE_CORP_COUNT_num  FINANCE_ORG_COUNT_max  FINANCE_ORG_COUNT_min  \
0                     0.0                 0.0000               0.000000   
1                     0.5                 0.

In [11]:
ceshidata.Y.value_counts()

0    28077
1     1874
Name: Y, dtype: int64

In [8]:
# from imblearn.over_sampling import SMOTE, ADASYN
# X_resampled, y_resampled = SMOTE(kind='borderline1').fit_sample(ceshidata.iloc[:,1:-1], ceshidata.iloc[:,-1])

In [12]:

# print(X_resampled[0:10,:])
# print(X_resampled.shape)
# print(y_resampled[0:10])
# print(y_resampled.shape)
# pd.Series(y_resampled).value_counts()
# x_train,x_valid,y_train, y_valid= train_test_split(X_resampled,y_resampled,test_size = 0.3)
x_train,x_valid,y_train, y_valid= train_test_split(ceshidata.iloc[:,1:-1],ceshidata.iloc[:,-1],test_size = 0.3)
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(20965, 61)
(20965,)
(8986, 61)
(8986,)


In [11]:
pd.Series(y_train).value_counts()

0    19626
1     1321
Name: Y, dtype: int64

In [13]:
# x_train = ceshidata.iloc[0:18000,1:-1]
# y_train = ceshidata.iloc[0:18000,-1]
# x_valid = ceshidata.iloc[18000:,1:-1]
# y_valid = ceshidata.iloc[18000:,-1]

skf = StratifiedKFold(n_splits=5)
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=3,
        scale_pos_weight=1,
        StratifiedKFold = True,
        seed=27)
xgb_param = xgb1.get_xgb_params()
#构建稀疏矩阵，运行更快
xgtrain = xgb.DMatrix(x_train, label = y_train)
# xgtest =xgb.DMatrix(dtest[predictors].values)
mm = xgb.cv(xgb_param, xgtrain, num_boost_round=1000, nfold=3,
    metrics='auc', early_stopping_rounds=50)
mm

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.603487,0.008997,0.628685,0.010569
1,0.615395,0.009274,0.646967,0.006068
2,0.614308,0.007758,0.659710,0.009753
3,0.619708,0.011067,0.673583,0.005484
4,0.620611,0.010902,0.681761,0.003828
5,0.622430,0.011649,0.691212,0.002374
6,0.622046,0.010877,0.694978,0.004989
7,0.620583,0.011486,0.699826,0.006229
8,0.620206,0.011526,0.706926,0.005880
9,0.618668,0.012228,0.712347,0.004607


In [14]:

print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(20965, 61)
(20965,)
(8986, 61)
(8986,)


In [15]:
feature = 0
for i in range(5):
    xgb2 = XGBClassifier(
            learning_rate =0.1,
            n_estimators=mm.shape[0],
            max_depth=5,
            min_child_weight=1,
            gamma=0,
            subsample=0.8,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,   #15:1
            StratifiedKFold = True,
            seed=27)
    xgb2.fit(x_train,y_train)
    feature += xgb2.feature_importances_
    print(i)
#     pred = xgb2.predict_proba(x_valid)[:,1]
#     print(pred)
#     auc = roc_auc_score(y_valid, pred)  
#     auc
feature = feature/5

0
1
2
3
4


In [16]:
feature

array([0.00317125, 0.00422833, 0.00211416, 0.00211416, 0.        ,
       0.        , 0.00845666, 0.00211416, 0.00211416, 0.00105708,
       0.        , 0.04016913, 0.01585624, 0.00951374, 0.00422833,
       0.        , 0.03699788, 0.02219873, 0.02008457, 0.00105708,
       0.        , 0.02536998, 0.02008457, 0.02536998, 0.        ,
       0.        , 0.02642706, 0.04651163, 0.04122622, 0.        ,
       0.        , 0.02008457, 0.02854123, 0.01691332, 0.        ,
       0.        , 0.03171247, 0.02008457, 0.0179704 , 0.        ,
       0.        , 0.04439746, 0.03911205, 0.02008457, 0.        ,
       0.        , 0.0359408 , 0.04756871, 0.03805497, 0.        ,
       0.        , 0.05285413, 0.03171247, 0.0422833 , 0.        ,
       0.        , 0.06131078, 0.04968287, 0.04122622, 0.        ,
       0.        ], dtype=float32)

In [17]:
print(x_train.columns)
print(feature[feature>0.0000008])
x_train.columns[feature>0.0000008]

Index(['TYPE_DW_num', 'FINANCE_CORP_COUNT_max', 'FINANCE_CORP_COUNT_min',
       'FINANCE_CORP_COUNT_mean', 'FINANCE_CORP_COUNT_zongshu',
       'FINANCE_CORP_COUNT_num', 'FINANCE_ORG_COUNT_max',
       'FINANCE_ORG_COUNT_min', 'FINANCE_ORG_COUNT_mean',
       'FINANCE_ORG_COUNT_zongshu', 'FINANCE_ORG_COUNT_num',
       'ACCOUNT_COUNT_max', 'ACCOUNT_COUNT_min', 'ACCOUNT_COUNT_mean',
       'ACCOUNT_COUNT_zongshu', 'ACCOUNT_COUNT_num', 'CREDIT_LIMIT_max',
       'CREDIT_LIMIT_min', 'CREDIT_LIMIT_mean', 'CREDIT_LIMIT_zongshu',
       'CREDIT_LIMIT_num', 'MAX_CREDIT_LIMIT_PER_ORG_max',
       'MAX_CREDIT_LIMIT_PER_ORG_min', 'MAX_CREDIT_LIMIT_PER_ORG_mean',
       'MAX_CREDIT_LIMIT_PER_ORG_zongshu', 'MAX_CREDIT_LIMIT_PER_ORG_num',
       'MIN_CREDIT_LIMIT_PER_ORG_max', 'MIN_CREDIT_LIMIT_PER_ORG_min',
       'MIN_CREDIT_LIMIT_PER_ORG_mean', 'MIN_CREDIT_LIMIT_PER_ORG_zongshu',
       'MIN_CREDIT_LIMIT_PER_ORG_num',
       'MAX_CREDIT_LIMIT_PER_ORGCutMIN_CREDIT_LIMIT_PER_ORG_max',
       'MAX

Index(['TYPE_DW_num', 'FINANCE_CORP_COUNT_max', 'FINANCE_CORP_COUNT_min',
       'FINANCE_CORP_COUNT_mean', 'FINANCE_ORG_COUNT_max',
       'FINANCE_ORG_COUNT_min', 'FINANCE_ORG_COUNT_mean',
       'FINANCE_ORG_COUNT_zongshu', 'ACCOUNT_COUNT_max', 'ACCOUNT_COUNT_min',
       'ACCOUNT_COUNT_mean', 'ACCOUNT_COUNT_zongshu', 'CREDIT_LIMIT_max',
       'CREDIT_LIMIT_min', 'CREDIT_LIMIT_mean', 'CREDIT_LIMIT_zongshu',
       'MAX_CREDIT_LIMIT_PER_ORG_max', 'MAX_CREDIT_LIMIT_PER_ORG_min',
       'MAX_CREDIT_LIMIT_PER_ORG_mean', 'MIN_CREDIT_LIMIT_PER_ORG_max',
       'MIN_CREDIT_LIMIT_PER_ORG_min', 'MIN_CREDIT_LIMIT_PER_ORG_mean',
       'MAX_CREDIT_LIMIT_PER_ORGCutMIN_CREDIT_LIMIT_PER_ORG_max',
       'MAX_CREDIT_LIMIT_PER_ORGCutMIN_CREDIT_LIMIT_PER_ORG_min',
       'MAX_CREDIT_LIMIT_PER_ORGCutMIN_CREDIT_LIMIT_PER_ORG_mean',
       'BALANCE_max', 'BALANCE_min', 'BALANCE_mean', 'USED_CREDIT_LIMIT_max',
       'USED_CREDIT_LIMIT_min', 'USED_CREDIT_LIMIT_mean',
       'CREDIT_LIMITCutUSED_CREDIT_

In [18]:
fulldata.head()
fulldataok = []
fulldataok = fulldata[x_train.columns[feature>0.0000008]]  #一定要小心是谁的ID
fulldataok = pd.concat([fulldata.REPORT_ID,fulldataok],axis = 1)
fulldataok.head()

,REPORT_ID,TYPE_DW_num,FINANCE_CORP_COUNT_max,FINANCE_CORP_COUNT_min,FINANCE_CORP_COUNT_mean,FINANCE_ORG_COUNT_max,FINANCE_ORG_COUNT_min,FINANCE_ORG_COUNT_mean,FINANCE_ORG_COUNT_zongshu,ACCOUNT_COUNT_max,...,USED_CREDIT_LIMIT_mean,CREDIT_LIMITCutUSED_CREDIT_LIMIT_max,CREDIT_LIMITCutUSED_CREDIT_LIMIT_min,CREDIT_LIMITCutUSED_CREDIT_LIMIT_mean,CREDIT_LIMITCutBALANCE_max,CREDIT_LIMITCutBALANCE_min,CREDIT_LIMITCutBALANCE_mean,LATEST_6M_USED_AVG_AMOUNT_max,LATEST_6M_USED_AVG_AMOUNT_min,LATEST_6M_USED_AVG_AMOUNT_mean
0,8787.0,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.006369,...,0.021444,0.015162,0.287575,0.029826,0.002373,0.013965,0.004585,0.011383,0.022856,0.022456
1,9410.0,0.5,0.800000,0.333333,0.666667,0.7500,0.333333,0.666667,0.0,0.114650,...,0.116274,0.019645,0.316406,0.036787,0.028517,0.069436,0.039047,0.080776,0.014485,0.086791
2,22999.0,1.0,0.333333,0.000000,0.166667,0.3125,0.000000,0.166667,0.0,0.057325,...,0.076650,0.016468,0.289719,0.030964,0.018201,0.006608,0.013548,0.080259,0.001050,0.056327
3,24820.0,0.0,0.466667,0.583333,0.583333,0.4375,0.583333,0.583333,0.0,0.057325,...,0.135202,0.020723,0.349399,0.040767,0.018037,0.105185,0.034853,0.074479,0.149544,0.146927
4,25708.0,0.5,0.200000,0.000000,0.083333,0.1875,0.000000,0.083333,0.0,0.031847,...,0.019033,0.015324,0.288609,0.030077,0.002974,0.000692,0.002964,0.031653,0.001233,0.031827


In [19]:
fulldataok.to_csv("../add/contest_ext_crd_is_sharedebtok.csv",index = False)

In [20]:
fulldataok.shape

(39948, 41)